<a href="https://colab.research.google.com/github/lakatosgabor/magyar_modszer/blob/main/Magyar_m%C3%B3dszer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import numpy as np
from scipy.optimize import linear_sum_assignment
import random


def get_operators():
  #itt kérdezem le a bent lévő opeártorokat
  operators = ["Gipsz Jakab", "Mona Lisa", "Teszt Elek"]
  return operators

def get_products():
  #itt kérdezem le, hogy mely termékeket kell gyártani
  #ha egy termék több operátoros az többször kerül be a listába
  products = ["A", "B", "C"]
  return products

def get_efficiency_matrix(operators, products):
  #itt kell lekérdezni az operátorok hatékonyságát az adott termékre
  EFFICIENCY = np.array([
    [200, 100, 100],
    [100, 100, 100],
    [200, 100, 100],
  ])

  EFFICIENCY = EFFICIENCY.astype(float)

  EFFICIENCY = to_square_matrix(EFFICIENCY)

  return EFFICIENCY

def to_square_matrix(matrix):
    rows, cols = matrix.shape
    if rows == cols:
        return matrix

    # Ha nem négyzetes, átalakítjuk
    n = max(rows, cols)
    square_matrix = np.zeros((n, n), dtype=matrix.dtype)
    square_matrix[:rows, :cols] = matrix
    return square_matrix

def get_monotony_matrix(operators, products):
  #ide jön a monotonítás lekérdezése

  monotony = np.array([
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1],
  ])

  monotony = monotony.astype(float)

  return to_square_matrix(monotony)

def pertubation(matrix):
    rows, cols = matrix.shape
    for col in range(cols):
        for row in range(rows):
            # Alapérték átalakítás és kis véletlenszerű eltérés hozzáadása, egy megadott tartományban generál kis véletlen eltéréseket
            matrix[row, col] = ((matrix[row, col] // 3) * 3) + np.random.uniform(-0.5, 0.5)
    return matrix

def to_constraints_matrix(operators, products, EFFICIENCY):
  #adatbázisból kérdezzül le
  constraints_matrix = np.array([
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1],
  ])

  constraints_matrix = constraints_matrix.astype(float)

  constraints_matrix = to_square_matrix(constraints_matrix)

    # Megkeressük az összes 0 értéket a constraints_matrix-ban
  zero_indices = np.where(constraints_matrix == 0)

  # Az azonos indexű elemeket a H mátrixban inf-re állítjuk
  EFFICIENCY[zero_indices] = np.inf

  return EFFICIENCY

def cost_matrix(EFFICIENCY):
  max_value = np.max(EFFICIENCY)
  EFFICIENCY = max_value - EFFICIENCY

  return EFFICIENCY

def run_hungarian_algorithm(matrix):
  row_ind, col_ind = linear_sum_assignment(EFFICIENCY)
  assignments = [(operators[i], products[j]) for i, j in zip(row_ind, col_ind)]

  return assignments


############### RUN PROCESS ###############

operators = get_operators()

products = get_products()

EFFICIENCY = get_efficiency_matrix(operators, products)

MONOTONY = get_monotony_matrix(operators, products)

EFFICIENCY = EFFICIENCY * MONOTONY

EFFICIENCY = pertubation(EFFICIENCY)

EFFICIENCY = cost_matrix(EFFICIENCY)

EFFICIENCY = to_constraints_matrix(operators, products, EFFICIENCY)

assignments = run_hungarian_algorithm(EFFICIENCY)
assignments

[('Gipsz Jakab', 'C'), ('Mona Lisa', 'B'), ('Teszt Elek', 'A')]